In [1]:
'''
Brute-Force匹配器：cv2.BFMatcher(normType, crossCheck)

normType - 指定要使用的距离量度,默认是cv2.NORM_L2,对于二进制字符串的描述子，比如ORB，BRIEF，BRISK等，应该用cv2.NORM_HAMMING；
crossCheck - 默认是false，如果它是true，匹配器返回那些和(i, j)匹配的，这样集合A里的第i个描述子和集合B里的第j个描述子最匹配，两个集合里的两个特征应该互相匹配，它提供了连续的结果。
matches = bf.match(des1, des2)的结果是DMatch对象列表

这个DMatch对象有下面的属性：

DMatch.distance - 描述子之间的距离。越低越好
DMatch.trainIdx - 训练描述子里的描述子索引
DMatch.queryIdx - 查询描述子里的描述子索引
DMatch.imgIdx - 训练图像的索引
'''

import cv2
import matplotlib.pyplot as plt
box = cv2.imread("FAU.jpg")
box_in_sence = cv2.imread("FAU1.jpg")

# 创建ORB特征检测器
orb = cv2.ORB_create()
kp1, des1 = orb.detectAndCompute(box,None)
kp2, des2 = orb.detectAndCompute(box_in_sence,None)

# 暴力匹配 汉明距离匹配特征点
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1,des2)

# 绘制匹配
result = cv2.drawMatches(box, kp1, box_in_sence, kp2, matches, None)
cv2.imwrite("orb-match.jpg", result)


True

In [3]:
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
# knn match
matches = bf.knnMatch(des1, des2, k=1)

# 删除matches里面的空list，并且根据距离排序
while [] in matches:
    matches.remove([])
matches = sorted(matches, key = lambda x:x[0].distance)

# 画出距离最短的前15个点
result = cv2.drawMatchesKnn(box, kp1, box_in_sence, kp2, matches[0:15], None, matchColor = (0,255,0), singlePointColor = (255,0,255))
cv2.imwrite("knn-match.jpg", result)

True

In [4]:

'''
FLANN是快速最近邻搜索包（Fast_Library_for_Approximate_Nearest_Neighbors）的简称，它是一个对大数据集和高维特征进行最近邻搜索的算法的集合，而且这些算法都已经被优化过了，在面对大数据集是它的效果要好于BFMatcher。
FLANN是一种高效的数值或者字符串匹配算法，SIFT/SURF是基于浮点数的匹配，ORB是二值匹配，速度更快。
对于FLANN匹配算法，当使用ORB匹配算法的时候，需要重新构造HASH。
flann = cv.FlannBasedMatcher(index_params, search_params)

indexParams - 字典，包含各种算法，具体可参考FLANN文档。
SearchParams - 字典，用来指定递归遍历的次数。值越高结果越准确，但是消耗的时间也越多，修改可以传入参数： search_params=dict( checks = 10)
'''

# 创建SIFT特征检测器
sift = cv2.xfeatures2d.SIFT_create()

kp1, des1 = sift.detectAndCompute(box,None)
kp2, des2 = sift.detectAndCompute(box_in_sence,None)


index_params = dict(algorithm = 0, trees = 5)

search_params = dict(checks=20)

flann = cv2.FlannBasedMatcher(index_params,search_params)

matches = flann.knnMatch(des1, des2, k=2)


# 记录好的点
goodMatches = [[0,0] for i in range(len(matches))]

for i,(m,n) in enumerate(matches):
    if m.distance < 0.7*n.distance:
        goodMatches[i]=[1,0]

draw_params = dict(matchColor = (0,255,0), singlePointColor = (255,0,0), matchesMask = goodMatches, flags = 0)

result = cv2.drawMatchesKnn(box, kp1, box_in_sence, kp2, matches, None, **draw_params)
cv2.imwrite("flann-match.jpg", result)

True